In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

print(os.listdir("./input"))

['BBC News Test.csv', 'BBC News Train.csv', 'fig_data_topicmodel.csv']


In [2]:
# Data Extraction and exploration

fig_data = pd.read_csv("./input/fig_data_topicmodel.csv")

In [3]:
fig_data.head()

,url_cleaned
0,droit-finances.commentcamarche.com/faq/15494-s...
1,droit-finances.commentcamarche.com/forum/affic...
2,droit-finances.commentcamarche.com/faq/4843-aa...
3,droit-finances.commentcamarche.com/faq/7527-al...
4,premium.lefigaro.fr/musique/2018/08/01/03006-2...


In [4]:
data_text = fig_data[['url_cleaned']]
data_text['index'] = data_text.index
documents = data_text

In [5]:
print(len(documents))
print(documents[:5])

70329
                                         url_cleaned  index
0  droit-finances.commentcamarche.com/faq/15494-s...      0
1  droit-finances.commentcamarche.com/forum/affic...      1
2  droit-finances.commentcamarche.com/faq/4843-aa...      2
3  droit-finances.commentcamarche.com/faq/7527-al...      3
4  premium.lefigaro.fr/musique/2018/08/01/03006-2...      4


In [6]:
!pip install gensim

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deepashree/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
#All stopwords are removed.
#Words that have fewer than 3 characters are removed.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

def make_bigrams(data_words):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    #trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)

    # See bigram example
    print(bigram_mod[data_words[0]])
    return [bigram_mod[doc] for doc in data_words]

In [8]:
processed_docs = documents['url_cleaned'].map(preprocess)
processed_docs[:10]

# Form Bigrams
processed_docs2 = make_bigrams(processed_docs)

['droit', 'finances', 'commentcamarche', 'salaire', 'assistante', 'maternelle', 'faut_savoir']


In [17]:
#key is the word and value is the number of times that word occurs in the entire corpus
dictionary = gensim.corpora.Dictionary(processed_docs2)
print(len(dictionary))

35545


In [18]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 20% of all documents
'''
dictionary.filter_extremes(no_below=3, no_above=0.20, keep_n= 100000)
print(len(dictionary))

15354


In [19]:
#bag of words
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs2]


In [22]:
#lda model
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [21]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.087*"info" + 0.046*"photos" + 0.041*"evenements" + 0.024*"sante" + 0.019*"mode" + 0.018*"prenoms" + 0.016*"scoop" + 0.015*"enfants" + 0.014*"prenom" + 0.012*"mode_shopping"


Topic: 1 
Words: 0.181*"linternaute" + 0.043*"actualite" + 0.030*"television" + 0.027*"magazine" + 0.023*"societe" + 0.022*"plus" + 0.020*"pour" + 0.016*"auto" + 0.015*"images" + 0.012*"personnalites"


Topic: 2 
Words: 0.123*"artfig" + 0.046*"actualite" + 0.037*"france" + 0.032*"politique" + 0.020*"international" + 0.019*"macron" + 0.019*"video" + 0.017*"scan" + 0.016*"societe" + 0.014*"premium"


Topic: 3 
Words: 0.116*"cuisine" + 0.071*"meteo" + 0.068*"recettes" + 0.064*"gastronomie" + 0.046*"france" + 0.035*"lachainemeteo" + 0.034*"recette" + 0.033*"previsions" + 0.022*"ville" + 0.007*"avec"


Topic: 4 
Words: 0.163*"people" + 0.118*"magazine" + 0.112*"stars" + 0.061*"infos" + 0.059*"actu" + 0.057*"news" + 0.056*"scoops" + 0.025*"leparticulier" + 0.025*"jcms" + 0.025*"week_chez"


Topic: 5 


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)